In [6]:
import requests, pymysql, math, re, time, csv
from bs4 import BeautifulSoup

In [13]:
companycode=list()
with open("companycode.csv",'r') as f:
    reader = csv.reader(f)
    
    for code in reader:
        companycode.append(code[0])

In [65]:
def crawl(code):
    
    url = "https://thevc.kr/"
    
    html = ""
    req = requests.get(url+code)
    if req.status_code == 200 :
        html = req.text

    soup = BeautifulSoup(html, 'html.parser')
    print("brand_list [HTTP Status : {}]".format(req))
    
    brand = []    
    brd_code = code
    brd_link = str()
    service_link = str()
    service_name = str()
    service_info = str()
    service_detail = str()
    
    for info in soup.find_all("div", class_="contact_info info_box"):
        for span in info.find_all("span"):
            href = span.find("a").get("href")
            if href[0:4] == "http":
                brd_link = href
    
    for prdt in soup.find_all("div", class_="info_box product_box"):
        print(prdt)
#         rank = inner.find("p", class_="n-label label-default txt_num_rank").text
#         brand.append(["brd_rank", str(rank[:rank.index("위")].strip())])
#         brand.append(["brd_code", inner.find("p", class_="brand_name").find("a").get("href")[22:].strip()])
#         brand.append(["brd_name_eng", inner.find("p", class_="brand_name_en").text.strip()])

#         dict_brand = dict(brand)
        
#         sql = "INSERT INTO BRAND_RANK(BRD_CODE, BRD_NAME_ENG, BRD_RANK) VALUES("\
#                      + "'" + dict_brand.get("brd_code", '오류') +"'" +  ","\
#                      + "'" + dict_brand.get("brd_name_eng", '오류') +"'" +  ","\
#                      + "'" + dict_brand.get("brd_rank", '오류') +"'" + ");"
        
#         print(sql)
    
#     brand = [brd_code, brd_link, service_link, service_name, service_info, service_detail]
    
    return brd_code, brd_link

In [66]:
crawl('ChannelCorporation')

brand_list [HTTP Status : <Response [200]>]
<div class="info_box product_box">
<a class="product_link" href="https://channel.io/ko" style="background-image: url(https://d1vv4p3ixxyri0.cloudfront.net/layout/profile_url.png)" target="_blank"></a>
<div class="img_wrapper">
<a href="https://channel.io/ko" onclick="ga('send','event','회사프로필/대시보드/서비스_제품','회사프로필/대시보드/서비스_제품/클릭')" target="_blank">
<img alt="채널톡" data-category="product" onerror="this.src='https://d1vv4p3ixxyri0.cloudfront.net/product_logo/default_product_logo.png'" src="https://d1vv4p3ixxyri0.cloudfront.net/product_logo/9e81b5928c09991c7e1273db6eb4611c7f07fc2e7ed259dc63de03ad02b82e6e_1556155910513277.jpg"/>
</a>
</div>
<div class="info_wrapper">
<a href="https://channel.io/ko" onclick="ga('send','event','회사프로필/대시보드/서비스_제품','회사프로필/대시보드/서비스_제품/클릭')" target="_blank">
<span class="product_name">채널톡 </span>
<span class="product_name"> (Channel Talk)</span>
</a>
<span class="product_name" style="-webkit-text-fill-color: #38d544;"> (운영

('ChannelCorporation', 'https://channel.io')